In [1]:
import numpy as np
import pandas as pd
import json
import pandas_market_calendars as mcal
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
from utils.preprocessing import generate_pair_data
from utils.finance_metrics import annualized_return

In [2]:
pairs = {eval(k): v for k,v in dict(json.loads(open("data/info/pairs.json", "r").read())).items()}
pairs

{('ADX', 'TY'): 0.853938041834032,
 ('ED', 'SO'): 0.7630684994843583,
 ('AEG', 'ING'): 0.7714520036919487,
 ('GOLD', 'NEM'): 0.8219942702537909,
 ('AVB', 'EQR'): 0.8793959812778221,
 ('AMAT', 'KLAC'): 0.8120695844768887,
 ('APA', 'DVN'): 0.7797792484401138,
 ('ARW', 'AVT'): 0.7573572791543869,
 ('CMA', 'TFC'): 0.8089975734507149,
 ('GFI', 'HMY'): 0.7721582544936532,
 ('BAC', 'WFC'): 0.8033690173669585,
 ('BBVA', 'SAN'): 0.8696850470469416,
 ('HIW', 'KRC'): 0.8127703586697915,
 ('BEN', 'TROW'): 0.7533682263460231,
 ('BK', 'NTRS'): 0.7853258900728711,
 ('HAL', 'SLB'): 0.7707796197283248,
 ('BMO', 'RY'): 0.8112242300303579,
 ('E', 'TOT'): 0.8145755214453303,
 ('BXP', 'VNO'): 0.8658210823728937,
 ('CCL', 'RCL'): 0.7796060510803956,
 ('CM', 'TD'): 0.7955914439701682,
 ('CVX', 'XOM'): 0.8257217118360992,
 ('CPT', 'UDR'): 0.8316230952071983,
 ('CSX', 'NSC'): 0.7742776958878398,
 ('CUZ', 'WRE'): 0.7571898754103825,
 ('DHI', 'LEN'): 0.8014581824672958,
 ('DRE', 'PLD'): 0.8245693410742715,
 ('EL

In [3]:
pair = ('ADX', 'TY')
df_A = pd.read_pickle("data/raw/" + pair[0] + ".zip")  # needs pickle5 compression (python 3.8)
df_B = pd.read_pickle("data/raw/" + pair[1] + ".zip")  # needs pickle5 compression (python 3.8)

pair_df = generate_pair_data(df_A, df_B, "1999-01-01", "2021-03-01")

pair_df.loc["2000-01-01":, :]

,1W Std.,1M Std.,3M Std.,1D Simple Ret.,1D Ret.,1W Ret.,1M Ret.,3M Ret.,1D Vol Ratio,1W Vol Ratio,1M Vol Ratio,3M Vol Ratio,1D GBM Proj.,1W GBM Proj.,1M GBM Proj.,3M GBM Proj.,Return Diff (t+1)
2000-01-03,0.858397,0.773041,1.050669,0.019197,0.018945,0.011071,0.006367,0.002754,0.555742,0.317096,0.233443,0.326796,-0.139307,-0.049903,-0.198165,0.026347,0.019197
2000-01-04,1.088119,0.783751,1.074592,-0.001656,-0.001635,0.008792,0.005883,0.003034,0.898808,0.384485,0.254607,0.329870,-0.166500,0.106437,0.057961,0.234076,-0.001656
2000-01-05,1.023830,0.831225,1.083355,-0.000625,-0.000621,0.006522,0.005936,0.003020,0.760250,0.469975,0.287690,0.335911,-0.041480,0.061364,-0.091600,-0.048785,-0.000625
2000-01-06,0.795298,0.738002,1.052704,-0.018349,-0.018519,0.004389,0.004765,0.002658,0.313247,0.473399,0.307057,0.333969,0.084399,-0.020840,-0.167687,0.016781,-0.018349
2000-01-07,0.743933,0.747537,1.049318,-0.003031,-0.003047,0.002021,0.003893,0.002574,0.252034,0.498786,0.314605,0.332667,-0.145277,-0.121995,-0.020924,0.058948,-0.003031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-23,1.225154,1.238262,1.471214,0.000167,0.000167,-0.000889,-0.000342,-0.001090,2.649200,1.447228,1.991273,2.336000,0.308996,-0.243938,-0.094251,-0.452310,0.000167
2021-02-24,1.361912,1.252530,1.120482,0.003574,0.003612,-0.001254,-0.000208,-0.000289,2.703189,1.855525,1.964926,2.303158,0.005430,0.119638,-0.236814,-0.027923,0.003574
2021-02-25,1.451676,1.299508,1.159543,-0.008334,-0.008196,-0.001283,-0.000587,-0.000302,1.356529,1.768606,1.920683,2.284094,-0.110348,-0.086563,-0.062112,0.183099,-0.008334
2021-02-26,1.437548,1.324815,1.161508,-0.002763,-0.002757,-0.001911,-0.000419,-0.000372,4.414030,1.942165,1.967836,2.247079,0.147805,-0.221466,-0.275143,0.175150,-0.002763


In [4]:
model = ElasticNet()
model.fit(np.array(pair_df.loc["2000-01-01":"2014-12-31", :].drop(columns="Return Diff (t+1)")),
          np.array(pair_df.loc["2000-01-01":"2014-12-31", "Return Diff (t+1)"]))
pred_diff = model.predict(np.array(pair_df.loc["2015-01-01":, :].drop(columns="Return Diff (t+1)")))
len(pred_diff)

1550

In [5]:
len(np.array(pair_df.loc["2015-01-01":, "Return Diff (t+1)"]))

1550

In [6]:
print(np.std(np.array(pair_df.loc["2000-01-01":"2014-12-31", "Return Diff (t+1)"])))
print(np.mean(np.array(pair_df.loc["2000-01-01":"2014-12-31", "Return Diff (t+1)"])))
print(np.min(np.array(pair_df.loc["2000-01-01":"2014-12-31", "Return Diff (t+1)"])))
print(np.max(np.array(pair_df.loc["2000-01-01":"2014-12-31", "Return Diff (t+1)"])))
mean_squared_error(np.array(pair_df.loc["2015-01-01":, "Return Diff (t+1)"]), pred_diff)

0.006816927387703143
3.948789574456831e-05
-0.04171816924864856
0.08396763204999211


2.5710565029871527e-05

In [7]:
annualized_return(df_A.loc[:,["Simple Return", "Log Return"]])

Simple Return    0.032908
Log Return       0.051147
dtype: float64

In [8]:
from utils.models import GeometricBrownianMotion

GeometricBrownianMotion(1, 0.03, 0.05).simulate()

array([0.99172931, 0.99277831, 1.00432389, 1.00674076, 1.00791104,
       1.00920625, 1.01170531, 1.00923052, 1.01228828, 1.01298921,
       1.01341368, 1.0146155 , 1.01862228, 1.01809335, 1.01724561,
       1.01753237, 1.01851956, 1.02091567, 1.02009161, 1.01887973,
       1.02210662, 1.02224956, 1.02846336, 1.03120447, 1.03448362,
       1.03902695, 1.04072878, 1.04401982, 1.04217382, 1.0406446 ,
       1.03790714, 1.03257674, 1.03519408, 1.02789047, 1.01955013,
       1.01889829, 1.01837971, 1.0263424 , 1.02304301, 1.02286041,
       1.03188977, 1.03134004, 1.0259098 , 1.01900912, 1.02110168,
       1.01991892, 1.02012838, 1.01256984, 1.01321823, 1.01020126,
       0.99605154, 0.99000317, 0.99644204, 0.99254304, 0.98793434,
       0.98596007, 0.99045282, 0.99238864, 0.99185128, 0.98704943,
       0.98536924, 0.98267561, 0.99124774, 0.99329436, 0.99479053,
       0.99052255, 0.99378918, 0.99733117, 0.99736811, 0.99460588,
       0.99851575, 0.99762597, 0.9938016 , 0.99142411, 0.99631